In [239]:
import pandas as pd

In [240]:
#import data csv
spa_data = pd.read_csv('finalspadata.csv')
rest_data = pd.read_csv('finaldata.csv')  
hotel_data = pd.read_csv('finalhoteldata.csv')

In [241]:
#convert price to numeric value
def cleandata(data):
    data = data.drop(columns={'Unnamed: 0', 'BusinessType'})
    data['price']=data['price'].apply(lambda x: 1 if (x == "€") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "€€") else x)
    data['price']=data['price'].apply(lambda x: 3 if (x == "€€€") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "££") else x)
    data['price']=data['price'].apply(lambda x: 1 if (x == "$") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "$$") else x)
    data['price']=data['price'].apply(lambda x: 3 if (x == "$$$") else x)
    data['price']=data['price'].apply(lambda x: 4 if (x == "$$$$") else x)
    data['price'] = data['price'].astype('int')
    
    #drop 0 values from database
    data = data.loc[data['price'] != 0]
    
    return data


In [242]:
#call function to clean data
spa_data = cleandata(spa_data)
rest_data = cleandata(rest_data)
hotel_data = cleandata(hotel_data)

In [243]:
#make a seperate database of expensive and cheap for business type for seperate analysis
exp_spa_data = spa_data.loc[spa_data['price'] >= 3]
cheap_spa_data = spa_data.loc[spa_data['price'] < 3]
exp_rest_data = rest_data.loc[rest_data['price'] >= 3]
cheap_rest_data = rest_data.loc[rest_data['price'] < 3]
exp_hotel_data = hotel_data.loc[hotel_data['price'] >= 3]
cheap_hotel_data = hotel_data.loc[hotel_data['price'] < 3] 

In [244]:
#read in PCI and pop data, clean and sort
other_db = pd.read_csv("final_before_yelp.csv")
other_db = other_db.drop(columns={'Unnamed: 0'})
other_db = other_db.rename(columns={'DataValue':'PCI'})
other_db = other_db.sort_values('State')
other_db.reset_index(drop=True, inplace=True)
other_db['City'] = other_db['City'].apply(lambda x: x.lower())
other_db['State'] = other_db['State'].apply(lambda x: x.lower())
other_db['PCI'] = other_db['PCI'].apply(lambda x: x.replace(',', ''))
other_db['PCI'] = other_db['PCI'].astype('int')


In [245]:
#function that creates a count of each price point and merges all databases

def buscount(data):
    data['City'] = data['City'].apply(lambda x: x.strip())
    data['State'] = data['State'].apply(lambda x: x.strip())
    data_means = data.groupby(['State', 'City']).mean()
    count = data.groupby(['State', 'City']).count()['name']
    data_count = pd.DataFrame(count)
    data_count.rename(columns={'name':'P total'}, inplace=True)
    data1 = data_count.merge(data_means, on=['State', 'City'], how='inner')
    data1['Price perc'] = data1['P total'] / data1['Total']    
    data1.reset_index(inplace=True)
    
    final = other_db.merge(data1, how='inner', on=['State', 'City'])
    return final

In [247]:
exp_spa_data = buscount(exp_spa_data)
cheap_spa_data = buscount(cheap_spa_data)
exp_rest_data = buscount(exp_rest_data)
cheap_rest_data = buscount(cheap_rest_data)
exp_hotel_data = buscount(exp_hotel_data)
cheap_hotel_data= buscount(cheap_hotel_data)


C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [251]:
exp_spa_data.to_csv('exp_spa_data.csv')
cheap_spa_data.to_csv('cheap_spa_data.csv')
exp_rest_data.to_csv('exp_rest_data.csv')
cheap_rest_data.to_csv('cheap_rest_data.csv')
exp_hotel_data.to_csv('exp_hotel_data.csv') 
cheap_hotel_data.to_csv('cheap_hotel_data.csv')

In [36]:
high_pop = final_db[(final_db['Pop'] > 60000)]
low_pop = final_db[(final_db['Pop'] < 60000)]


In [61]:
high_pop = high_pop.merge(data1_means, how='inner', on=['State', 'City'])
low_pop = low_pop.merge(data1_means, how='inner', on=['State', 'City'])

In [64]:
hp_f = final_db.merge(hp_final, how='inner', on=['State', 'City'])

In [252]:
final_final = final_db.merge(data1_means, how='inner', on=['State', 'City'])

In [62]:
high_pop.to_csv('high_pop.csv')
low_pop.to_csv('low_pop.csv')